In [ ]:
!pip install llama_index datasets

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
import os

os.environ["OPENAI_API_KEY"] = ''
# 추후 사용할 llm, 임베딩 모델 클래스 정의
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
!wget "https://www.dropbox.com/scl/fi/ywc29qvt66s8i97h1taci/lyft-10k-2020.pdf?rlkey=d7bru2jno7398imeirn09fey5&dl=0" -q -O ./lyft_10k_2020.pdf
!wget "https://www.dropbox.com/scl/fi/lpmmki7a9a14s1l5ef7ep/lyft-10k-2021.pdf?rlkey=ud5cwlfotrii6r5jjag1o3hvm&dl=0" -q -O ./lyft_10k_2021.pdf
!wget "https://www.dropbox.com/scl/fi/iffbbnbw9h7shqnnot5es/lyft-10k-2022.pdf?rlkey=grkdgxcrib60oegtp4jn8hpl8&dl=0" -q -O ./lyft_10k_2022.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
#naive engine 구성
lyft_2020_docs = SimpleDirectoryReader()
lyft_2020_index = VectorStoreIndex.from_documents()
lyft_2020_engine =

lyft_2021_docs = SimpleDirectoryReader()
lyft_2021_index = VectorStoreIndex.from_documents()
lyft_2021_engine =

lyft_2022_docs = SimpleDirectoryReader()
lyft_2022_index = VectorStoreIndex.from_documents()
lyft_2022_engine =

In [ ]:
response = lyft_2022_engine.query()
print(response)


 - Agent가 활용할 수 있는 툴들을 정의 필요
 - 방금 만든 쿼리 엔진들을 툴화 시키면 됨.
 - 툴 화 시킬때는 각 툴들이 언제 사용되어야 하는지를 자연어로 태깅 필요.
 - 그런 다음 LLM(에이전트)이 해당 자연어를 토대로 classification 수행.

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_2020_engine,
        metadata=ToolMetadata(
            name="lyft_2020_10k_form",
            description="Annual report of Lyft's financial activities in 2020",
        ),
    ),
    QueryEngineTool(
        query_engine=lyft_2021_engine,
        metadata=ToolMetadata(
            name="lyft_2021_10k_form",
            description="Annual report of Lyft's financial activities in 2021",
        ),
    ),
    QueryEngineTool(
        query_engine=lyft_2022_engine,
        metadata=ToolMetadata(
            name="lyft_2022_10k_form",
            description="Annual report of Lyft's financial activities in 2022",
        ),
    ),
]



- 이제 agent를 생성시켜줄 차례
- agent가 스탭바이스탭으로 쿼리를 더 심플한 구조로 변환(트랜스폼)해서 자료를 찾은 다음, 모아서 원래의 복잡한 쿼리에 대한 답안으로 도출시킴

In [ ]:
from llama_index.core.agent import ReActAgent

#agent 생성
lyft_agent = ReActAgent.from_tools(query_engine_tools, verbose=True)
response = lyft_agent.chat("")
print(response)

In [ ]:
#질문 : What is Lyft's profit in 2022 vs 2020? Generate only one step at a time. Use existing tools.
response = lyft_agent.chat("")
print(response)

In [ ]:
#질문: Lyft의 2022년과 2020년의 경영상황이 어떻게 되?
response = lyft_agent.chat("")
print(response)

In [ ]:
response = lyft_agent.chat("What did Lyft do in R&D in 2022 versus 2021? Generate only one step at a time. Use existing tools.")
print(response)

In [ ]:
response = lyft_agent.chat("2020년은?")
print(response)

# DO IT YOURSELF
- korean_webtext 문서를 활용해서 문서별 queryengine 정의
- react agent가 담당 쿼리엔진을 잘 찾을 수 있도록 llm 활용하여 metadata generation 해보기
- 이후 테스트 해보면서 메타데이터 생성 프롬프트 수정해보기

In [ ]:
# 파일 로드
import pandas as pd
data = pd.read_csv('/content/korean_webtext.csv').iloc[:,1:]



In [ ]:
# 메타데이터 생성용 llm
llm = OpenAI(model="gpt-4o-mini")

In [ ]:

data['topic'] = data['text'].apply(lambda x: llm.complete(
    f"""Given the text,
    <여기에 프롬프트를 입력해 보세요>
    text:{x}
    description:
    """))

In [ ]:
data

In [ ]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
documents = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    documents.append(Document(
        text=row['text'],
        extra_info={'topic': row['topic'].text}
    ))

In [ ]:
query_engine_tools = []
i = 0
for doc in documents:
    i += 1
    index = VectorStoreIndex.from_documents(documents=[doc])
    engine = index.as_query_engine()
    query_engine_tools.append(
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name=f"tool_{i}",
                description=f"{doc.extra_info['topic']}와 관련된 내용이 있는 문서")))


In [ ]:
#ReActAgent 생성된 쿼리 엔진 툴스에 연결
wiki_agent =


# 이제 아래 쿼리들에 대해 답안을 받아보시고, 결과가 잘 안나오면 위로 다시 돌아가서 메타데이터 생성 방식을 바꿔보세요!

In [ ]:
response = wiki_agent.chat("분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까? Always consider one of the existing tools to answer the question.")
print(response)

In [ ]:
response = wiki_agent.chat("이번에는 캐나다가 주목받는 국가인 이유에 대해 알려줘. Always consider one of the existing tools to answer the question.")
print(response)

In [ ]:
response = wiki_agent.chat("한국과 일본의 유명한 해군 제독에 대해 알려줘. Always consider one of the existing tools to answer the question.")
print(response)

In [ ]:
response = wiki_agent.chat("프랭키가 누구한테 인기가 없을것 같아? Always consider one of the existing tools to answer the question.")
print(response)

In [ ]:
response = wiki_agent.chat("프랭키가 누구한테 인기가 있을것 같아? Always consider one of the existing tools to answer the question.")
print(response)

In [ ]:
response = wiki_agent.chat("동양권 국가와 서양권 국가간의 전쟁에 대해 알려줘. Always consider one of the existing tools to answer the question.")
print(response)